# Ollama + OpenAI + Python

## 1. Specificare il nome del modello

Se hai importato un modello diverso da "phi3:mini", modifica il valore nella cella qui sotto.  
Questa variabile verrà utilizzata nel codice in tutto il notebook.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Configurare il client Open AI

Di solito, il client OpenAI viene utilizzato con OpenAI.com o Azure OpenAI per interagire con modelli linguistici di grandi dimensioni.  
Tuttavia, può essere utilizzato anche con Ollama, poiché Ollama offre un endpoint compatibile con OpenAI all'indirizzo "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generare un completamento della chat

Ora possiamo utilizzare l'SDK di OpenAI per generare una risposta per una conversazione. Questa richiesta dovrebbe generare un haiku sui gatti:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Progettazione dei prompt

Il primo messaggio inviato al modello linguistico è chiamato "messaggio di sistema" o "prompt di sistema", e stabilisce le istruzioni generali per il modello.  
Puoi fornire un tuo prompt di sistema personalizzato per guidare un modello linguistico a generare output in un modo diverso.  
Modifica il `SYSTEM_MESSAGE` qui sotto per rispondere come il tuo personaggio preferito di un famoso film/serie TV, oppure trova ispirazione per altri prompt di sistema da [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Una volta personalizzato il messaggio di sistema, fornisci la prima domanda dell'utente nel `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Esempi con pochi esempi

Un altro modo per guidare un modello linguistico è fornire "pochi esempi", una sequenza di domande/risposte che dimostrano come dovrebbe rispondere.

L'esempio qui sotto cerca di far comportare un modello linguistico come un assistente didattico fornendo alcuni esempi di domande e risposte che un assistente potrebbe dare, e poi propone al modello una domanda che uno studente potrebbe fare.

Prova prima questo approccio, e poi modifica il `SYSTEM_MESSAGE`, `EXAMPLES` e `USER_MESSAGE` per un nuovo scenario.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Generazione Aumentata da Recupero

RAG (Generazione Aumentata da Recupero) è una tecnica che permette a un modello linguistico di rispondere accuratamente a domande relative a un determinato dominio, recuperando prima informazioni pertinenti da una fonte di conoscenza e poi generando una risposta basata su tali informazioni.

Abbiamo fornito un file CSV locale con dati sui veicoli ibridi. Il codice qui sotto legge il file CSV, cerca corrispondenze con la domanda dell'utente e genera una risposta basata sulle informazioni trovate. Nota che questo processo richiederà più tempo rispetto agli esempi precedenti, poiché invia più dati al modello. Se noti che la risposta non è ancora basata sui dati, puoi provare a fare ingegneria del sistema o utilizzare altri modelli. In generale, RAG è più efficace con modelli più grandi o con versioni ottimizzate di SLM.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
